In [1]:
library(FigR)
library(Seurat)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(foreach)

Loading required package: Matrix

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats,

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/pairs.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/matrix.atac.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/matrix.rna.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/FigR/BMMC22.merge.250211/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac_count = readRDS(path.matrix.atac_count)
matrix.atac = centerCounts(matrix.atac_count)

Matrix object input detectedCentering counts for cells sequentially in groups of size  1000  ..

Computing centered counts for cells:  1  to  1000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  1001  to  2000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  2001  to  3000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  3001  to  4000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  4001  to  5000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  5001  to  6000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  6001  to  7000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  

Import RNA matrix

In [6]:
matrix.rna_count = readRDS(path.matrix.rna_count)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna)
rm(matrix.rna_count)

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:SummarizedExperiment’:

    shift


The following object is masked from ‘package:GenomicRanges’:

    shift


The following object is masked from ‘package:IRanges’:

    shift


The following objects are masked from ‘package:S4Vectors’:

    first, second




In [9]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                              pairs.E2G$PeakName %in% rownames(matrix.atac)]

Prepare FigR input data

In [10]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak$PeakName = paste(seqnames(bed.peak),
                          start(bed.peak),
                          end(bed.peak),
                          sep = "-")
names(bed.peak) = bed.peak$PeakName
bed.peak = bed.peak[rownames(matrix.atac)]

In [11]:
ATAC.se = SummarizedExperiment(assays = SimpleList(counts = matrix.atac_count),
                               rowRanges = bed.peak)
rm(matrix.atac_count)
ATAC.se <- chromVAR::addGCBias(ATAC.se, genome = BSgenome.Hsapiens.UCSC.hg38)
ATAC.se

class: RangedSummarizedExperiment 
dim: 540429 69249 
metadata(0):
assays(1): counts
rownames(540429): chr1-10010-10536 chr1-96496-96699 ...
  chrX-156029831-156030380 chrX-156030497-156030869
rowData names(2): PeakName bias
colnames(69249): s1d1_TAGTTGTCACCCTCAC-1 s1d1_CTATGGCCATAACGGG-1 ...
  s4d9_ACACTTGCAACTAGAA-1 s4d9_CACTTAAAGTCTGGGC-1
colData names(0):

In [12]:
bg <- chromVAR::getBackgroundPeaks(ATAC.se, niterations = 100)

In [13]:
index.gene = data.frame(gene_name = rownames(matrix.rna.rename),
                       index = 1:nrow(matrix.rna.rename))
rownames(index.gene) = index.gene$gene_name

index.peak = data.frame(peak_name = rownames(matrix.atac),
                       index = 1:nrow(matrix.atac))
rownames(index.peak) = index.peak$peak_name

Run FigR prediction

In [14]:
PeakGeneCor.modified = function (ATAC, RNA, OV, ncores = 4, chunkSize = 200, metric = "spearman", 
    bg = NULL) 
{
    stopifnot(ncol(ATAC) == ncol(RNA))
    if (chunkSize > 1000) 
        stop("Do not specify very large chunk sizes. Please use chunkSize < 1000")
    # n <- length(OV)
    n <- nrow(OV) # import OV as a dataframe instead of Hits object
    starts <- seq(1, n, chunkSize)
    ends <- starts + chunkSize - 1
    ends[length(ends)] <- n
    # OVd <- OV %>% as.data.frame() %>% dplyr::rename(Gene = "queryHits", 
    #     Peak = "subjectHits")
    OVd <- OV %>% dplyr::rename(Gene = "queryHits", Peak = "subjectHits")
    chunkList <- mapply(c, starts, ends, SIMPLIFY = FALSE)
    time_elapsed <- Sys.time()
    cat("Running in parallel using ", ncores, "cores ..\n")
    cat("Computing observed correlations ..\n")
    corList <- pbmcapply::pbmclapply(X = chunkList, FUN = function(x) {
        FigR::chunkCore(chunk = x, A = ATAC, R = RNA, O = OVd, 
            met = metric)
    }, mc.cores = ncores)
    if (any(unlist(sapply(corList, is.null)))) {
        message("One or more of the chunk processes failed unexpectedly (returned NULL) ..")
        message("Please check to see you have enough cores/memory allocated")
        message("Also make sure you have filtered down to non-zero peaks/genes")
    }
    OVd$rObs <- unlist(corList)
    cat("Finished!\n")
    time_elapsed <- Sys.time() - time_elapsed
    cat(paste("\nTime Elapsed: ", time_elapsed, units(time_elapsed)), 
        "\n\n")
    if (!is.null(bg)) {
        n_iter <- ncol(bg)
        cat("Computing background correlations ..\n")
        time_elapsed <- Sys.time()
        bgCor <- foreach(i = 1:n_iter, .combine = "cbind", .export = c("chunkCore", 
            "t"), .packages = c("pbmcapply", "FigR", "Matrix")) %do% 
            {
                OVdBg <- OVd[, 1:2]
                OVdBg$Peak <- bg[OVdBg$Peak, i]
                bgCorList <- pbmcapply::pbmclapply(X = chunkList, 
                  FUN = function(x) {
                    chunkCore(chunk = x, A = ATAC, R = RNA, O = OVdBg, 
                      met = metric)
                  }, mc.cores = ncores)
                unlist(bgCorList)
            }
        if (sum(is.null(bgCor)) != 0 | sum(is.na(bgCor)) != 0) 
            stop("One or more of the chunk processes failed unexpectedly (returned NULL) .. Please check to see you have enough cores/m\n           emory allocated")
        time_elapsed <- Sys.time() - time_elapsed
        cat(paste("\nTime Elapsed: ", time_elapsed, units(time_elapsed)), 
            "\n\n")
        colnames(bgCor) <- paste0("rBg", 1:ncol(bgCor))
        OVd <- cbind(OVd, bgCor)
    }
    return(OVd)
}

In [15]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

Warning message in dir.create(paste(dir.output, "chr", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/FigR/BMMC22.merge.250211//chr' already exists”


In [16]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter)))
chr.run = chr.run[!chr.run %in% chr.done]

In [ ]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter[seqnames(pairs.E2G.filter) == chr.tmp]
  genePeakOv.chr <- data.frame(queryHits = index.gene[pairs.E2G.chr.res$TargetGene, "index"],
                               subjectHits = index.peak[pairs.E2G.chr.res$PeakName, "index"])
  ObsCor.chr = PeakGeneCor.modified(ATAC = matrix.atac, 
                                    RNA = matrix.rna,
                                    OV = genePeakOv.chr, 
                                    chunkSize = 500,
                                    ncores = n.cores, 
                                    bg = bg)
  pairs.E2G.chr.res$rObs = ObsCor.chr[,"rObs"]
  pairs.E2G.chr.res$rBgSD <- matrixStats::rowSds(as.matrix(ObsCor.chr[, 4:103]))
  pairs.E2G.chr.res$rBgMean <- rowMeans(ObsCor.chr[, 4:103])
  pairs.E2G.chr.res$pvalZ <- 1 - stats::pnorm(q = pairs.E2G.chr.res$rObs, 
                                              mean = pairs.E2G.chr.res$rBgMean,
                                              sd = pairs.E2G.chr.res$rBgSD)
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr13"
Running in parallel using  16 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  54.867161989212 secs 

Computing background correlations ..


Loading required package: parallel



Merge results

In [ ]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [ ]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

In [ ]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "rObs",
                         "rBgSD",
                         "rBgMean",
                         "pvalZ")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

In [ ]:
sessionInfo()